<img src='img/revival_hero_image-dan-meyers-unsplash.png'>

<small>Photo by Dan Meyers for Unsplash</small>

# Revival = Survival: A data project about the overdose crisis
by Shawn Syms
<br>
<br>

<a name="contents"></a>
## Contents

* <a href="#executive-summary">Executive summary</a>
* <a href='#data-visualization'>Module 1: Data visualization: Learning about the overdose crisis in Canada and Toronto</a>
* <a href='#predictive-model'>Module 2: Training a machine learning model to predict overdose outcomes</a>
* <a href='#services-locator'>Module 3: Creating a simple tool to point users to the locations of recovery resources</a>

> * <a href="#technical-configuration">Technical configuration</a>
> * <a href="#data-acquisition">Data acquisition</a>
> * <a href="#data-cleaning-and-eda">Data cleaning, exploratory data analysis (EDA) and preprocessing</a>
> * <a href="#user-experience">User experience</a>


<a name="executive-summary"></a>
## Executive summary

The purpose of this project is three-fold. First, I use data visualization tools and techniques to tell the story of the current state of the overdose epidemic nationally and locally. Second, I have trained a machine-learning model to try and predict overdose outcomes given relevant data. Third, I've built a simple tool based on multiple geo-spatial datasets where users can access resources to help stop overdoses, including pharmacies where they can access naloxone. Carrying it on their person, anyone can save a life.

<div style="text-align: right">(<a href="#contents">home</a>) </div>

<a name="data-visualization"></a>
## Module 1: Learning about the overdose crisis in Canada and Toronto through data visualization

An interactive version of these visualizations is available online at:

> <a href='https://public.tableau.com/profile/shawn.syms#!/vizhome/shared/2DNT56KHS'>Tableau Public: Shawn Syms</a>

<strong>SUMMARY:</strong> I review multiple datasets with national, province and local information to see what they reveal about the opioid-overdose crisis here at home. 

### Key Data Points
1. The opioid overdose crisis is a growing issue here in Canada. 

<img src='img/revival-data_visualization_1.png'>

<br><br>
2. Across Ontario, drugs with an astonishing toxicity are available.

<img src='img/revival-data_visualization_2.png'>

<br><br>
3. As Fentanyl enters the drug trade, more and more lives are lost.

<img src='img/revival-data_visualization_3.png'>

<br><br>
4. Deaths start to taper when Safer Injection Facilities arrive. 

<img src='img/revival-data_visualization_4.png'>

<br><br>
5. But there is more we can do to save lives, using Naloxone. 

<img src='img/revival-data_visualization_5.png'>

<div style="text-align: right">(<a href="#contents">home</a>) </div>

<a name="services-locator"></a>
## Module 2: Overdose resources locator

<strong>SUMMARY:</strong> I write a basic python script to validate a user's location and identify the nearest overdose-related resources, placing them on a map. These resources include: pharmacies where the opiate-antidote naloxone can be obtained, safer injection sites as well as medical clinics focused on addiction recovery. 

For the moment, this is a python script in a Jupyter notebook, but the concept could be expanded to a mobile site such that user could check in real time which overdose prevention resources are in nearest proximity to them and their device.

<img src='img/landing_page.png'>

<a name="technical-configuration"></a>
## Technical configuration
### Environment set-up

This app was set up using anaconda and a virtual environment. To mimic it, run the commands below from a command prompt. If you don't have anaconda, you can use pip to achieve the same results.

In [217]:
## Anaconda virtual environment; run from a terminal window
#
# conda create --name shopify flask  numpy pandas requests 
#
# From inside the newly created environment:
# conda install -c conda-forge folium

### Imports
<a name="imports-and-functions"></a>

In [402]:
# The following packages should be imported in order for the application to work correctly.

import folium
import math
 
import requests
import time

from flask import Flask, render_template
from folium.plugins import MarkerCluster
from IPython.display import HTML

# %xmode Minimal
# %xmode Plain
# %xmode Context
# %xmode Verbose 


### Functions

In [403]:
'''Here is a list of all functions used by the application.
* by Shawn Syms, September 2020
'''

def geocode_ip():
    
    '''This function uses the GeoJS API to identify or approximate 
    * a user's location NOTE: This is based on IP address, which
    * reduces accuracy
    '''
    
    get_ip = requests.get('https://get.geojs.io/v1/ip.json') # request user's IP address
    ip_address = get_ip.json()['ip']                         # parse via json
    location_request = requests.get('https://get.geojs.io/v1/ip/geo/' + ip_address + '.json') 
    # request additional params
    location_parameters = location_request.json()            # parse via json 
    user_longitude = location_parameters['longitude']
    user_latitude = location_parameters['latitude']
    return (user_longitude, user_latitude)


def individual_postal_code_to_lat_long(postcode):
    
    ''' This function uses the GeoCoder.ca API to get a pair of 
    * lat/long coordinates by looking up the postal code
    
    * Geocoding by IP address is quite inaccurate, so I am considering
    * alternative approaches
    '''
    
    location_request = requests.get('https://geocoder.ca/?postal=' + postcode + '&geoit=XML&json=1')
    location_parameters = location_request.json()
    latitude = location_parameters['latt']
    longitude = location_parameters['longt']
    return latitude, longitude


def bulk_postal_code_to_lat_long(dataframe):
    
    '''This function using the geocoder.ca API to get longitude and
    * latitude values for a list of observations by supplying the 
    * postal code. Assumes that every observation includes a valid 
    * postal code
    
    * Note: In my testing I found the geocoder.ca lat long data
    * to be pretty inaccurate. I ended up using a service called "Geocoding
    * by Awesome Table, which uses the Google Maps API and Google Sheets.
    * Keeping this function here for posterity though."
    '''
    
    for i in range(len(dataframe)):  # for each observation
        # format a request in the manner that the Geocoder API expects
        location_request = requests.get('https://geocoder.ca/?postal=' + dataframe['Postcode'][i] + '&geoit=XML&json=1')
        location_parameters = location_request.json() # Get details in json format
        latitude = location_parameters['latt']
        longitude = location_parameters['longt']
        
        try:
            dataframe['latitude'][i] = latitude
            
        except:
            dataframe['latitude'][i] = np.nan
                    
        try:
            dataframe['longitude'][i] = longitude
            
        except:
            dataframe['longitude'][i] = np.nan
            
        time.sleep(5)


def distance(origin_lat, origin_long, dest_lat, dest_long):
    
    ''' The Haversine formula is a mathematical equation for finding the distance 
    * between two points on the globe given that it is spherical and not flat.
    * 
    * The original author of this implementation is WAYNE DYCK. I edited it 
    * to better suit my objectives.
    * 
    * NOTE: This is distance "as the crow flies" -- a next step
    * would be to if I can get use Google APIs to give distance 
    * by the shortest travel route.
    '''

    earths_radius = 6371 # km

    dlat = math.radians(dest_lat - origin_lat)
    dlon = math.radians(dest_long - origin_long)
    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(origin_lat)) \
        * math.cos(math.radians(dest_lat)) * math.sin(dlon/2) * math.sin(dlon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = earths_radius * c

    return d


def isolate_the_variable(pandas_series):
    
    '''This simple function takes a pandas series representing the contents 
    of a dataframe column, and isolates the first value. It strips out some 
    extraneous values that are part of the series.'''
    
    return pandas_series.tolist()[0]

def exploratory_data_analysis(df):
    
    '''This function outputs some basic EDA information
    about a pandas dataframe'''
    
    # set some variables for EDA outputs
    columns = df.dtypes
    described = df.describe()
    head = df.head()
    tail = df.tail()
    duplicates = df[df.duplicated()]

    
    # output the values
    
    print('Here is the columns data:')
    print(columns)
    print('')
    print('Here are the numeric components of your dataset, described:')
    print(described)
    print('')
    print('Here are the first five observations:')
    print(head)
    print('')
    print('Here are the final five observations:')
    print(tail)
    print('')
    print('Here are any duplicate observations:')
    print(duplicates)
    
def data_cleaning(df):
    
    '''This function applies some data hygiene to a
    given dataset. My original pharmacy dataset was quite messy and needed this.
    The replacement one, from the Ontario Pharmacists Association, is
    quite clean and didn't need it. Keeping for posterity.'''
    
    # change to Upper/lower case; use a list comprehension
    for columns in df.columns:
        df[columns] = df[columns].str.title() 

    # get rid of any pharmacies with identical addresses
    df.drop_duplicates(subset ="Address", keep = False, inplace = True)
    
    # establish proper formatting for postal code 
    # assumes the space is missing
    
    df['Postcode'] = [str(i[:3] + ' ' + i[3:]) for i in df['Postcode']]

def user_experience():
    
    '''This function takes the user name and what the user is looking for,
    * looks up the user's location and compares it to the appropriate
    * dataset of recovery resources by geographic distance, and returns
    * the resource closest to the user, on a map
    '''
    
    user_name = input('Welcome! May I please have your first name? ')

    user_choice = input('What do you want to do today: '
                        '(1) Get Naloxone to save a life, '
                        '(2) Access recovery resources,' 
                        '(3) Access safer-injection facilities? ')
    
    # Distinguish between three types of resources and set variables
    if user_choice == '1':
        df = pd.read_csv('csv/locator/official_toronto_pharmacy_roster.csv')
        locator_type = "pharmacy"
        print("")
        print("Naloxone is available at your nearest pharmacy.")
        print("")
    elif user_choice == '2':
        df = pd.read_csv('csv/locator/toronto-raam_clinic_locations.csv')
        locator_type = "Rapid Access to Addiction Medicine clinic"
        print("")
        print("If you or someone you know is serious about recovery,")
        print("physicians at a rapid-access-to-addiction-medicine (RAAM)")
        print("clinic and help you today.")
        print("")

    elif user_choice == '3':
        df = pd.read_csv('csv/locator/toronto-safe_site_locations.csv')
        locator_type = 'safer injection facility'
        print("")
        print("People who inject drugs should ideally never do so alone.")
        print("Visit the nearest safer injection site.")
        print("")
    
    # default to the pharmacy locator if the input is ambiguous
    
    else:
        df = pd.read_csv('csv/locator/official_toronto_pharmacy_roster.csv')
        locator_type = "pharmacy"
        print("")
        print("Naloxone is available at your nearest pharmacy.")
        print("")

    # identify user's location by IP
    # Replace this with HMTL 5 Geolocation in JS/Leaflet/Flask version

    user_location = geocode()
    user_latitude = user_location[1]
    user_longitude = user_location[0]
    user_coordinates=str(user_location[1] + ',' + user_location[0])

    # Set up a dataframe with their information

    user_details = {'Name': [user_name],
            'Address': ['Current User Location'],
            'City': ['Current city'],
            'Postcode': ["Current location postal code"],
            'Latitude': [user_latitude],
            'Longitude': [user_longitude],
            'Distance': 0}

    user_data = pd.DataFrame(user_details)
    df = df.append(user_data) # append it to the dataframe in memory
    df = df.astype({'Longitude' : 'float', 'Latitude' : 'float'})
    df.reset_index(drop=True, inplace=True)

    # Set some variables related to the user
    user_index = df[df['Name']==user_name].index.values
    user_observation = df.loc[user_index]
    user_latitude = user_observation['Latitude'].tolist()[0] 
    user_longitude = user_observation['Longitude'].tolist()[0]
    user_coordinates = user_latitude, user_longitude
    
    ''' Loop through the dataset calculating distance from user
    * for each one. This solution scales to the city pharmacist
    * dataset with ease, but a different approach would be needed to 
    * scale up to provincial or national data
    '''

    for i in (range(len(df))):
        df['Distance'][i] = distance(user_observation['Latitude'],
                                  user_observation['Longitude'], 
                                  df['Latitude'][i],
                                  df['Longitude'][i])
    df.sort_values(by=['Distance'], inplace=True)
    df.reset_index(drop=True, inplace=True)
    
    # Identify the closest location
    closest_location = df.loc[1]
    closest_location_coordinates = closest_location['Latitude'], closest_location['Longitude']

    # Return the results
    print(user_name + ', your closest ' + locator_type + ' is ' + closest_location['Name'] + ', located at ' + closest_location['Address'] + '.')
    print('Here is a map showing your nearest ' + locator_type + ", " + user_name + '.')

    #Create the map
    locator_map = folium.Map(location = user_coordinates, zoom_start = 13)

    folium.Marker(user_coordinates, popup = 'You are here').add_to(locator_map)
    folium.Marker(closest_location_coordinates, popup = closest_location['Name']).add_to(locator_map)

    #Display the map
    return(locator_map)
        


<div style="text-align: right">(<a href="#contents">home</a>) </div>

<a name="data-acquisition"></a>
### Data acquisition

This app includes code that will do a geo-spatial lookup of support resources for people who want to help prevent drug overdoses. See the readme file for source locations for all relevant datasets. Copies of the datasets can be found in this Github repository in the csv directory. (Note: All of the datasets used in the data visualizations above can be found in that directory as well.) First, the datasets are explored and cleaned.

<div style="text-align: right">(<a href="#contents">home</a>) </div>

<a name="data-cleaning-and-eda"></a>
## Data cleaning, exploratory data analysis and preprocessing

### Pharmacies

Pharmacies across the province and around Ontario dispense Naloxone kits and offer training on their use. Let's have a look at our Toronto pharmacies dataset.

In [225]:
pharmacies = pd.read_csv('csv/locator/official_toronto_pharmacy_roster.csv')

exploratory_data_analysis(pharmacies)

Here is the columns data:
Name          object
Address       object
Phone         object
City          object
Postcode      object
Latitude     float64
Longitude    float64
Distance       int64
dtype: object

Here are the numeric components of your dataset, described:
         Latitude   Longitude  Distance
count  543.000000  543.000000     543.0
mean    43.683824  -79.398612       0.0
std      0.038630    0.066840       0.0
min     43.589969  -79.603678       0.0
25%     43.656498  -79.438192       0.0
50%     43.671491  -79.393801       0.0
75%     43.703384  -79.365980       0.0
max     43.835605  -79.185016       0.0

Here are the first five observations:
                      Name                Address           Phone     City  \
0       1ClinicRx Pharmacy            16 Yonge St  (647) 344-8800  Toronto   
1              99 Pharmacy       436 Dundas St. W  (416) 977-8338  Toronto   
2          Action Pharmacy        2425 Bloor St W  (416) 766-2401  Toronto   
3    Albion Finch Ph

<div style="text-align: right">(<a href="#contents">home</a>) </div>

### Toronto safer-injection sites

Safer-injection sites offer supervision in order to mitigate overdose risk. An app user may need this information for themselves or someone they know. Let's have a look at our Toronto dataset of safer-injection sites.

In [226]:
safe_sites = pd.read_csv('csv/locator/toronto-safe_site_locations.csv')

exploratory_data_analysis(safe_sites)

Here is the columns data:
Name          object
Address       object
City          object
Province      object
Postcode      object
Latitude     float64
Longitude    float64
Distance     float64
dtype: object

Here are the numeric components of your dataset, described:
        Latitude  Longitude  Distance
count   9.000000   9.000000       9.0
mean   43.654267 -79.381488       0.0
std     0.006294   0.026656       0.0
min    43.641824 -79.429466       0.0
25%    43.653650 -79.403172       0.0
50%    43.656426 -79.372785       0.0
75%    43.658689 -79.369567       0.0
max    43.661143 -79.339202       0.0

Here are the first five observations:
                                                Name                 Address  \
0               Fred Victor Overdose Prevention Site   145 Queen Street East   
1                 Moss Park Overdose Prevention Site   134 Sherbourne Street   
2  Parkdale Queen West Community Health Centre Ov...     168 Bathurst Street   
3                  Parkdale Ov

In [227]:
safe_sites

,Name,Address,City,Province,Postcode,Latitude,Longitude,Distance
0,Fred Victor Overdose Prevention Site,145 Queen Street East,Toronto,Ontario,M5A 1S1,43.653650,-79.372785,0.0
1,Moss Park Overdose Prevention Site,134 Sherbourne Street,Toronto,Ontario,M5A 4J4,43.654171,-79.369567,0.0
2,Parkdale Queen West Community Health Centre Ov...,168 Bathurst Street,Toronto,Ontario,M5V 2R4,43.646589,-79.404227,0.0
3,Parkdale Overdose Prevention Site,1229 Queen Street West,Toronto,Ontario,M6K 1L2,43.641824,-79.429466,0.0
4,Regent Park Overdose Prevention Site,465 Dundas Street East,Toronto,Ontario,M5A 2B2,43.659376,-79.365585,0.0
5,South Riverdale Community Health Centre,955 Queen Street East,Toronto,Ontario,M4M 3P3,43.661143,-79.339202,0.0
6,St. Stephen's Community House,260 Augusta Avenue,Toronto,Ontario,M5T 2L9,43.656426,-79.403172,0.0
7,Street Health,338 Dundas Street East,Toronto,Ontario,M5A 2A1,43.658689,-79.370007,0.0
8,The Works,277 Victoria Street,Toronto,Ontario,M5B 1W2,43.656536,-79.379377,0.0


<div style="text-align: right">(<a href="#contents">home</a>) </div>

### Toronto RAAM clinics

A Rapid Access to Addictive Medicine (RAAM) clinic offers medical treatment for problem substance use. An app user may need this information for themselves or someone they know. Let's have a look at our Toronto dataset of RAAM clinics.

In [228]:
raam_clinics = pd.read_csv('csv/locator/toronto-raam_clinic_locations.csv')

exploratory_data_analysis(raam_clinics)

Here is the columns data:
Name          object
Address       object
Phone         object
City          object
Province      object
Postcode      object
Latitude     float64
Longitude    float64
Distance       int64
dtype: object

Here are the numeric components of your dataset, described:
        Latitude  Longitude  Distance
count   9.000000   9.000000       9.0
mean   43.675460 -79.396372       0.0
std     0.030269   0.047768       0.0
min    43.640394 -79.488756       0.0
25%    43.653735 -79.405942       0.0
50%    43.664728 -79.385876       0.0
75%    43.689871 -79.374172       0.0
max    43.723780 -79.324858       0.0

Here are the first five observations:
                       Name                           Address  \
0      Unity Health Toronto        30 Bond Street, 17th floor   
1     Humber River Hospital     1235 Wilson Avenue, 5th floor   
2  Women's College Hospital    76 Grenville Street, 3rd floor   
3      Unity Health Toronto  410 Sherbourne Street, 1st floor   
4   

In [229]:
raam_clinics

,Name,Address,Phone,City,Province,Postcode,Latitude,Longitude,Distance
0,Unity Health Toronto,"30 Bond Street, 17th floor",416-864-6060 x2567,Toronto,Ontario,M5B 1X1,43.653735,-79.377398,0
1,Humber River Hospital,"1235 Wilson Avenue, 5th floor",1-888-399-8342,Toronto,Ontario,M3M 3G7,43.723780,-79.488756,0
2,Women's College Hospital,"76 Grenville Street, 3rd floor",416-323-7559 x6,Toronto,Ontario,M5S 1B2,43.661514,-79.387121,0
3,Unity Health Toronto,"410 Sherbourne Street, 1st floor",416-864-3096,Toronto,Ontario,M4X 1K2,43.664728,-79.374172,0
4,Unity Health Toronto,"30 The Queensway, Ground floor",416-530-6486 x3969,Toronto,Ontario,M6R 1B5,43.640394,-79.450347,0
5,University Health Network,"399 Bathurst Street, 1st floor,",416-726-5052,Toronto,Ontario,M5T 2S8,43.653386,-79.405942,0
6,Michael Garron Hospital,825 Coxwell Avenue,416-461-8272,Toronto,Ontario,M4C 3E7,43.689871,-79.324858,0
7,Sunnybrook Health Sciences Centre,"2075 Bayview Avenue, Room A146",416-480-6736,Toronto,Ontario,M4N 3M5,43.722846,-79.372875,0
8,Anishnawbe Health Toronto,"4 Charles Street East, 3rd floor",416-657-0379 x234,Toronto,Ontario,M4Y 1T2,43.668886,-79.385876,0


<div style="text-align: right">(<a href="#contents">home</a>) </div>

### Get geocoding data for our datasets

In [36]:
'''One option is to Iterate through the list and query for the longitude and latitude 
using the Geocoder API. Unhash the code below (and the subsequent cell) 
if you wish to run the process to gather the latitude and longitude coordinates. 

NOTE: I found the Geocoder API to not be very accurate. See the note in the function
for more details. The saved csv files have the longitude and latitude built into them
already'''

# bulk_postal_code_to_lat_long(pharmacies_to_process)

''' Save the transformed data
pharmacies_to_process.to_csv('csv/toronto_pharmacies_saved.csv', index = False)'''

In [37]:
''' Save the transformed data
pharmacies_to_process.to_csv('csv/toronto_pharmacies_saved.csv', index = False)'''

<div style="text-align: right">(<a href="#contents">home</a>) </div>

<a name="user-experience"></a>
## User experience

In [401]:
user_experience()

Welcome! May I please have your first name? shawn
What do you want to do today: (1) Get Naloxone to save a life, (2) Access recovery resources,(3) Access safer-injection facilities? 2

If you or someone you know is serious about recovery,
physicians at a rapid-access-to-addiction-medicine (RAAM)
clinic and help you today.

shawn, your closest Rapid Access to Addiction Medicine clinic is University Health Network, located at 399 Bathurst Street, 1st floor,.
Here is a map showing your nearest Rapid Access to Addiction Medicine clinic, shawn.


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


<img src='img/sample_mappng_large.png'>

<div style="text-align: right">(<a href="#contents">home</a>) </div>